In [1]:
import scipy.stats as sps
import scipy.optimize as opt
import scipy.integrate as integr
import numpy as np
import matplotlib.pyplot as plt

# Критерий согласия Пирсона (для сложной гипотезы)

In [2]:
# Константы задачи

xn = np.array([0]*5 + [1]*8 + [2]*6 + [3]*12 + [4]*14 + [5]*18 + [6]*11 + [7]*6 + [8]*13 + [9]*7)
n = xn.size
sqrt_n = n ** 0.5

m = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
k = m.size
s = 2

In [3]:
# Функции для подсчета логарифма функции правдоподобия 

def get_norm_pdf(x_cur, teta1_cur, teta2_cur):
    return sps.norm.pdf(x = x_cur, loc = teta1_cur, scale = teta2_cur)


def optim_log(teta_cur, pairs_cur):
    L = 1
    for i in range(len(pairs_cur[0])):
        integral_value = integr.quad(get_norm_pdf, pairs_cur[0][i], pairs_cur[1][i], args = (teta_cur[0], teta_cur[1]))
        L *= integral_value[0]
        
    return -np.log(L)


In [4]:
# Образуем интервалы

pairs = np.array([[[-np.inf]*5 + [1]*8 + [2]*6 + [3]*12 + [4]*14 + [5]*18 + [6]*11 + [7]*6 + [8]*13 + [9]*7] , 
                  [[1]*5 + [2]*8 + [3]*6 + [4]*12 + [5]*14 + [6]*18 + [7]*11 + [8]*6 + [9]*13 + [np.inf]*7]])
pairs = np.squeeze(pairs, axis=1)

print("/1/", n)
for i in range(len(pairs[0])//10):
    print(pairs[0][i], pairs[1][i])
print("...")

print("/2/", get_norm_pdf(-np.inf, 4, 3))
print("/3/", integr.quad(get_norm_pdf, -np.inf, 0, args = (4, 3))[0])
    
# Максимизируем логарифм (в данном случае, минимализируем -lnL)

optima = opt.minimize(fun = optim_log,  x0 = [5, 2], args = (pairs,)) # из графика выборки возьмем 5, 2*sigma (95%) = 4

my_teta1 = round(optima.x[0], 4)
my_teta2 = round(optima.x[1], 4)
print("/4/", "my_teta1, my_teta2 : ",my_teta1, my_teta2)

/1/ 100
-inf 1.0
-inf 1.0
-inf 1.0
-inf 1.0
-inf 1.0
1.0 2.0
1.0 2.0
1.0 2.0
1.0 2.0
1.0 2.0
...
/2/ 0.0
/3/ 0.09121121972586788
/4/ my_teta1, my_teta2 :  5.2897 2.6795


In [5]:
# Посчитаем n*pi:

my_p_full = np.array([integr.quad(get_norm_pdf, pairs[0][i], pairs[1][i], args = (my_teta1, my_teta2))[0] for i in range(n)])
my_np_full = my_p_full * n

my_np_res = np.delete(my_np_full, np.where(np.diff(my_np_full) == 0)[0] + 1)
my_np_res = np.round(my_np_res, 3) #убрал повторки соотвествующие одному mi и немного округлил
print("/5/", my_np_res)

# Посчитаем ~delta

my_delta = np.sum((m - my_np_res)**2 / my_np_res)
print("/6/", "MY_DELTA = ", my_delta)

/5/ [ 5.47   5.508  8.663 11.874 14.181 14.758 13.383 10.575  7.282  8.307]
/6/ MY_DELTA =  9.801418398650844


In [6]:
# Посчитаем p-value

p_value = 1 - sps.chi2.cdf(my_delta, k - 1 - s)
print("p-value", p_value)

p-value 0.20010899336247512


## p-value > alpha, где alpha = 0.05 - задали заранее. Как итог, нет оснований отвергать Ho.

# Критерий согласия Колмогорова (для сложной гипотезы)

In [7]:
# Эмпирическая функция распределения по выборке

def emp_func(sample_cur, n_cur):
    # Получаем точки - пары x и y координат 
    xy_pairs = np.vstack(( np.sort(sample_cur), np.arange(0, n_cur) / n_cur )).T
    uniq_elem = xy_pairs[0]
    for pair_cur in xy_pairs:
        if pair_cur[0] == uniq_elem[0]:
            pair_cur[1] = uniq_elem[1]
        else:
            uniq_elem = pair_cur
    return xy_pairs

In [8]:
# Функция распределения нормального закона

def norm_func(sample_cur, teta1_cur, teta2_cur):
    x1 = np.sort(sample_cur)
    y1 = sps.norm.cdf(x = x1, loc = teta1_cur, scale = teta2_cur)
    return np.vstack((x1, y1)).T

In [9]:
# Константы задачи

xn = np.array([0]*5 + [1]*8 + [2]*6 + [3]*12 + [4]*14 + [5]*18 + [6]*11 + [7]*6 + [8]*13 + [9]*7)
n = xn.size
sqrt_n = n ** 0.5

In [10]:
# Получаем из исходной выборки

my_teta1 = np.mean(xn)
my_teta2 = np.std(xn)
print("/1/", n, my_teta1, my_teta2)

my_empf = emp_func(xn, n) # ~Fn(x)
print("/2/", my_empf.shape)
print("/3/", my_empf[:10])

my_normf = norm_func(xn, my_teta1, my_teta2) # F(x, vector my_teta)
print("/4/", my_normf.shape)
print("/5/", my_normf[:10])

my_delta = sqrt_n * max(np.max(np.abs(my_empf[:, 1] - my_normf[:, 1])), np.max(np.abs(my_empf[1:, 1] - my_normf[:-1, 1])), np.abs(1 - my_normf[-1, 1])) # ~delta
print("/6/", np.abs(my_empf[:, 1] - my_normf[:, 1])[:10], np.abs(my_empf[1:, 1] - my_normf[:-1, 1])[:10], np.abs(1 - my_normf[-1, 1]))
print("/7/", "MY_DELTA = ", my_delta)


/1/ 100 4.77 2.505414137423193
/2/ (100, 2)
/3/ [[0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [1.   0.05]
 [1.   0.05]
 [1.   0.05]
 [1.   0.05]
 [1.   0.05]]
/4/ (100, 2)
/5/ [[0.         0.02846311]
 [0.         0.02846311]
 [0.         0.02846311]
 [0.         0.02846311]
 [0.         0.02846311]
 [1.         0.06619531]
 [1.         0.06619531]
 [1.         0.06619531]
 [1.         0.06619531]
 [1.         0.06619531]]
/6/ [0.02846311 0.02846311 0.02846311 0.02846311 0.02846311 0.01619531
 0.01619531 0.01619531 0.01619531 0.01619531] [0.02846311 0.02846311 0.02846311 0.02846311 0.02153689 0.01619531
 0.01619531 0.01619531 0.01619531 0.01619531] 0.045672642243731576
/7/ MY_DELTA =  1.013371112422481


In [11]:
# Воспользуемся параметрическим bootstapом

N = 50000
x_boot = np.array([(np.round(sps.norm.rvs(size = n, loc = my_teta1, scale = my_teta2)))%10 for i in range (N)])
print("/8/", x_boot.shape)

teta1_boot = np.mean(x_boot, axis=1)
teta2_boot = np.std(x_boot, axis=1)
print("/9/", teta1_boot.shape)
print("/10/", teta1_boot[:5])
print("/11/", teta2_boot[:5])

empf_boot = np.apply_along_axis(func1d = emp_func, axis = 1, arr = x_boot, n_cur = n) 
print("/12/", empf_boot.shape)
print("/13/", empf_boot[0][:10])


delta_boot = np.array([])
for i in range(N):
    empf_boot_cur = empf_boot[i] # ~F*n(x)
    normf_boot_cur = norm_func(x_boot[i], teta1_boot[i], teta2_boot[i]) # F(x, vector teta*)
    delta_boot_cur = sqrt_n * max(np.max(np.abs(empf_boot_cur[:, 1] - normf_boot_cur[:, 1])), 
                                  np.max(np.abs(empf_boot_cur[1:, 1] - normf_boot_cur[:-1, 1])), 
                                  np.abs(1 - normf_boot_cur[-1, 1])) # *delta
    delta_boot = np.append(delta_boot, delta_boot_cur)



delta_boot = np.sort(delta_boot) # вариационный ряд
print("/14/", delta_boot.shape)

l = len([delta for delta in delta_boot if delta >= my_delta])
print("/15/", l)

p_value = l / N
print("/16/", p_value)




/8/ (50000, 100)
/9/ (50000,)
/10/ [4.92 4.43 4.56 4.71 4.59]
/11/ [2.25246532 2.47893122 2.53897617 2.29910852 2.23201703]
/12/ (50000, 100, 2)
/13/ [[0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [1.   0.04]
 [1.   0.04]
 [1.   0.04]
 [1.   0.04]
 [1.   0.04]
 [2.   0.09]]
/14/ (50000,)
/15/ 43396
/16/ 0.86792


In [12]:
print("p-value = ", p_value)

p-value =  0.86792


## p-value > alpha, где alpha = 0.05 - задали заранее. 
## Тогда, как итог, нет оснований отвергнуть гипотезу H0

In [ ]:
#Пункт а  - КОЛМОГОРОВ 
xn = xn = np.array([0]*5 + [1]*8 + [2]*6 + [3]*12 + [4]*14 + [5]*18 + [6]*11 + [7]*6 + [8]*13 + [9]*7)
n = xn.size

emp_func_sample = np.array([]) # он будет содержать значения на полуинтервалах, а не только в точках
for i in range(n):
    emp_func_s_i = sum(1 for j in xn if j < xn[i]) / n
    emp_func_sample = np.append(emp_func_sample, emp_func_s_i)
emp_func_sample = np.append(emp_func_sample, 1)
emp_func_sample = np.array([*set(emp_func_sample)]) #убрал дубликаты
emp_func_sample = np.sort(emp_func_sample) #отсортировал

emp_func = np.array([]) # он будет содержать значения только в точках
for i in range(n):
    emp_func_i = sps.uniform.cdf(xn[i], 0, 9)
    emp_func = np.append(emp_func, emp_func_i)
emp_func = np.array([*set(emp_func)]) #убрал дубликаты
emp_func = np.sort(emp_func)

diff_emp_func = np.array([])
for i in range(len(emp_func)):
    diff_emp_func_i = max( np.abs(emp_func_sample[i] - emp_func[i]), np.abs(emp_func_sample[i+1] - emp_func[i]))
    diff_emp_func = np.append(diff_emp_func, diff_emp_func_i)

delta_sample = np.sqrt(n) * np.max(diff_emp_func)
print(delta_sample)
